![](https://mcd.unison.mx/wp-content/themes/awaken/img/logo_mcd.png)

# Curso Ingeniería de Características

## Haciendo mapas en python

### Julio Waissman

En esta pequeña libreta vamos a ver como utilizar folium para realizar mapas, con énfasis en hacer mapas de Sonora (aunque a partir de aqui, hacer mapas de diferentes partes del mundo ya es trivial.

Para los mapas, vamos a utilizar folium y plotly. [Folium](http://python-visualization.github.io/folium/index.html) es una interfase a la fabulosa biblioteca [leaflet](https://leafletjs.com) de Javascript.

Para instalar folium se puede hacer (inclusive en colab

```bash
$ pip install folium
```

y si tienes tu propio entorno de `conda` pues el clásico

```bash
$ conda install folium -c conda-forge

```


In [ ]:
!conda install folium -c conda-forge -y

## 1. Generacion de mapas y sus propiedades

Hacer mapas es ir agregando capas y capas sobre el mismo mapa con nueva información. Pero la primera capa debe ser siempre el mapa en si, y luego ya agregaremos marcadores, grupos de marcadores, cloropetos, datos y demás.

Para hacer mapas en *folium se utiliza la clase `Map`. El objeto generado es un mapa, el cual se puede visualizar, guardar o modificar. Veamos:

In [ ]:
# Creando un mapa sólo para ver:

import folium

m = folium.Map(
    location=(29.66667, -110.5),
    tiles='OpenStreetMap',
    zoom_start=7,
    min_zoom=0,
    max_zoom=18,
    width='80%',
    height='80'
)
m

Como vemos, se puede controlar la posición inicial del mapa, el tamaño, el zoom máximo, mínimo e inicial, y el tipo de mapa que se muestra. Vamos a probar diferentes tipos de mapas. Los mapas que vienen por default en *folium* son los siguientes:

- "OpenStreetMap"
- "CartoDB" (positron and dark_matter)
- "Stamen" (Terrain, Toner, and Watercolor)
- "Mapbox Bright" (Niveles limitados de zoom gratuitos)
- "Mapbox Control Room" (Niveles limitados de zoom gratuitos)
- "Cloudmade" (con costo)
- "Mapbox" (con costo)
 
Además se pueden agregar mapas de elaboración propia que sean iguales a los que soporta `leaflet.js`. En la práctica, con los mapas gratuitos tenemos suficiente en la gran mayoría de los casos.

Una capa muy útil para hacer más mapas es la que nos permite obtener las latitudes y longitudes de diferentes en diferentes puntos del mapa. Los que nos permite obtener valores puntuales donde poner marcadores, si es que lo queremos hacer a mano.



In [ ]:
m.add_child(folium.LatLngPopup())
m

## 2. Punto específicos o Marcadores

Para señalar lugares puntuales en un mapa se usan los marcadores, simplemente hay que agregar uno por uno, con la informacion necesaria en cada uno. Los marcadores tienen un string e texto para el `tooltip` y otro para el `popup`. Este texto puede estar formateado siempre y cuando se pase en formato html.

A los marcadores se les puede cambiar el ícono que muestra y el color del marcador usando los iconos de [glyphicon](https://getbootstrap.com/docs/3.3/components/) (no hay que poner prefijo, son por default) o con [font-awesome](https://fontawesome.com/v5.15/icons?d=gallery&p=2&m=free) (es necesario poner en el ícono `prefix='fa'`)

Para más información revisar la documentación de ambas clases.



In [ ]:
#folium.Marker?
#folium.Icon?

In [ ]:
m = folium.Map(
    location=(29.0809, -110.9605),
    tiles='Openstreetmap',
    zoom_start=13,
    min_zoom=12,
)

folium.Marker(
    location=[29.0809, -110.9605],
    tooltip="<b>Maestría en Ciencia de Datos</b><br>Coordinación",
    popup='<b>Maestría en Ciencia de Datos</b><br>Visitanos en <a href="mcd.unison.mx">nuestra página oficial</a>',
    icon=folium.Icon(icon="info-sign", color="red")
).add_to(m)

folium.Marker(
    location=[29.0836, -110.9581],
    tooltip="Biblioteca",
    popup='Este edificio bien podía ser ruso, de la época estalinista (o español en la época de Franco)',
    icon=folium.Icon(icon="book")
).add_to(m)

folium.Marker(
    location=[29.0932, -111.0220],
    tooltip="Aquí vamos a llenarnos de gloria"
).add_to(m)

m

In [ ]:
import pandas as pd

municipios = pd.read_csv("municipios_latlon.csv")
municipios_poblacion = pd.read_csv("POBLACION_SONORA2020.csv")

municipios = municipios.merge(municipios_poblacion, on="MUNICIPIO") 

municipios.info()

In [ ]:
m = folium.Map(
    location=(29.66667, -110.5),
    tiles='OpenStreetMap',
    zoom_start=7
)

municipios['POBLA'] = municipios['POBLACION TOTAL']

for row in municipios.itertuples():
    folium.Marker(
        location=[row.LATITUD, row.LONGITUD],
        tooltip=f"<b>{row.MUNICIPIO}</b><br>Población: {row.POBLA} habitantes",
    ).add_to(m)
m

Y tambien podemos hacerlo por grupos de marcadores (cuando son muchos los marcadores es práctico)

In [ ]:
from folium.plugins import MarkerCluster

m = folium.Map(
    location=(29.66667, -110.5),
    tiles='OpenStreetMap',
    zoom_start=7
)

grupo = MarkerCluster().add_to(m)

for row in municipios.itertuples():
    folium.Marker(
        location=[row.LATITUD, row.LONGITUD],
        tooltip=f"<b>{row.MUNICIPIO}</b><br>Población: {row.POBLA} habitantes",
    ).add_to(grupo)
m

## 3. Mapas coloropéticos

Los mapas coloropéticos son polígonos con los cuales se pueden representar regiones en un mapa. Hay muchos cloropetos diferentes, y es importante tener cloropetos sobre diferentes formas de representar polígonos, sobre todo en México y en Sonora.

Vamos a hacer uno en rápido y fusioso

In [ ]:
import json

with open("poligonos_sonora.geojson") as fp:
    municipios_geojson = json.load(fp)


In [ ]:
municipios_geojson

In [ ]:
municipios = []
for index in range(len(municipios_geojson['features'])):
    municipios.append(municipios_geojson['features'][index]['properties']['Municipio'])
municipios
